In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import neo
import neo_importers.neo_spike_importer as spike_importer
from neo_importers.neo_wrapper import MNGRecording
from features import FeatureDatabase

file_name = Path("..")/".."/"Files"/"latency_experiment_roberto"/"20_05_13_U1a_pulse_Latenz.smr"

# maybe we could make the spike importer return a recording object immediately?
# or recording + block representation?
bl, id_map = spike_importer.import_spike_file(file_name, 
                                      stimuli_event_channels={"DigMark"},
                                      action_potential_channels= "all"
                                      )
seg: neo.core.Segment = bl.segments[0]
recording = MNGRecording(bl.segments[0])

print("There are the following channels in this recording: " + str(recording.all_channels.keys()))

Units "°C" can not be converted to a quantity. Using dimensionless instead
Units "°C" can not be converted to a quantity. Using dimensionless instead


There are the following channels in this recording: dict_keys(['rd.0', 'rd.1', 'es.0', 'ap.0', 'ap.1', 'ap.2', 'ap.3', 'ap.4'])


In [20]:
from neo.core import AnalogSignal

rd_channel: AnalogSignal
for key, rd_channel in recording.raw_data_channels.items():
    print(key, rd_channel.name)

rd.0 Temp.
rd.1 Signal


In [2]:
%autoreload 2
from features import ResponseLatencyFeatureExtractor, NormalizedSignalEnergyExtractor, SpikeCountExtractor, AdaptiveSpikeCountExtractor
from quantities import second

# define feature db
db = FeatureDatabase(Path("../../Files/latency_experiment_roberto/20_05_13_U1a_pulse_Latenz_Features"), recording)

try: # check if we can just load the features somehow
    db.load()
    print("Found feature database and loaded from files...")
except Exception as ex: # extract the features from the data base
    ap_channels = list(recording.action_potential_channels.keys())
    db.extract_features(ap_channels, ResponseLatencyFeatureExtractor, stimulus_channel = "es.0")
    db.extract_features(ap_channels, NormalizedSignalEnergyExtractor)
    db.extract_features(ap_channels, SpikeCountExtractor, timeframe = 100 * second, num_intervals = 8)
    db.extract_features(ap_channels, AdaptiveSpikeCountExtractor, timeframe = 100 * second, num_intervals = 6)
    db.store()
    print("Created and stored new feature db...")

Found feature database and loaded from files...


In [26]:
%autoreload 2
from fibre_tracking import APTrack

raw_signal = recording.raw_data_channel_by_name(name = "Signal")
el_stimuli = recording.electrical_stimulus_channels["es.0"]
ap_channel = recording.action_potential_channels["ap.2"]

ap_track = APTrack.from_aps(el_stimuli = el_stimuli, aps = list(ap_channel[0 : 10]))
ap_track.color = "green"

print(ap_track)

AP Track with 10 latencies:
Starts at sweep 0 with latency 0.004640000000008559 s.


In [27]:
%autoreload 2
ap_track.extend_downwards(raw_signal = raw_signal, el_stimuli = el_stimuli, num_sweeps = 10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.72s/it]


In [29]:
%autoreload 2
from plotting import FallingLeafPlot

from quantities import second, minute

flp = FallingLeafPlot(recording = recording, feature_db = db)

flp.plot(action_potential_channels = "all", plot_raw_signal = True, analog_signal_channel = "rd.0", \
         t_start = 0 * minute, post_stimulus_timeframe = 0.02 * second, num_intervals = 25, 
         ap_tracks = [ap_track])

0.0 min 173.15185999999997 s
